# Analysis of the non-linear solvers

In [11]:
import numpy as np
import scipy as sp
import scipy.stats as stats
from averageTime import *
import math
import os
import pickle

In [4]:
# 1 is functional, 2 is Newton

def load_data():        
    # check whether the folder 'Benchmarking_of_numerical_ODE_integration_methods/Data' exists
    if not os.path.exists('../../Benchmarking_of_numerical_ODE_integration_methods/Data/WholeStudy'):
        base_path = '../Data/WholeStudy'
    elif os.path.exists('../../Benchmarking_of_numerical_ODE_integration_methods/Data/WholeStudy'):
        base_path = '../../Benchmarking_of_numerical_ODE_integration_methods/Data/WholeStudy'

    # list of all data frames for nonLinSol == 1 for better indexing in the future
    all_intern_columns_1 = [pd.DataFrame(columns=[]) for _ in range(70)]

    # list of all data frames for nonLinSol == 2 for better indexing in the future
    all_intern_columns_2 = [pd.DataFrame(columns=[]) for _ in range(70)]

    all_intern_columns = [all_intern_columns_1, all_intern_columns_2]
    column_names = []

    # choose only the correct files
    all_files = sorted(os.listdir(base_path))
    correct_files_1 = []
    correct_files_2 = []
    for iFile in range(0, len(all_files)):
        if all_files[iFile].split('_')[0] == '1':
            correct_files_1.append(all_files[iFile])
        elif all_files[iFile].split('_')[0] == '2':
            correct_files_2.append(all_files[iFile])
    correct_files = [correct_files_1, correct_files_2]

    # open all .tsv linear solver files + save right column in data frame
    for iNonLinSol in range(0, len(correct_files)):
        for iCorrectFile in range(0, len(correct_files_1)):
            next_tsv = pd.read_csv(base_path + '/' + correct_files[iNonLinSol][iCorrectFile], sep='\t')

            # change .tsv-id form e.g. 1_06_10.tsv to 06_10
            new_name = correct_files[iNonLinSol][iCorrectFile].split('.')[0].split('_')[3] + '_' + \
                       correct_files[iNonLinSol][iCorrectFile].split('.')[0].split('_')[4]

            # reset after each iteration
            next_time_value = []
            num_x = []

            # open next file
            next_tsv = averaging(next_tsv)

            # get the correct values
            for iFile in range(0, len(next_tsv['id'])):  # each file
                if next_tsv['t_intern_ms'][iFile] != 0:
                    next_time_value.append(next_tsv['t_intern_ms'][iFile])
                    num_x.append(next_tsv['state_variables'][iFile])

            # append new column to existing data frame with correct values
            column_names.append(str(new_name))
            all_intern_columns[iNonLinSol][iCorrectFile]['state_variables'] = pd.Series(num_x)
            all_intern_columns[iNonLinSol][iCorrectFile][str(new_name)] = pd.Series(next_time_value)

    # length of the last file
    file_length = len(next_tsv['id'])

    # initialize y-data
    adams_data_1 = []
    bdf_data_1 = []
    adams_data_2 = []
    bdf_data_2 = []

    # Functional
    blank_space_counter = 0
    for iDensityPoint in range(0, int(len(correct_files_1)/2) + 4):
        if iDensityPoint in [7, 15, 23, 31]:
            adams_data_1.append(math.inf)
            bdf_data_1.append(math.inf)
            blank_space_counter += 1
        else:
            adams_data_1.append(1 - round(len(all_intern_columns_1[
                iDensityPoint - blank_space_counter][column_names[iDensityPoint - blank_space_counter]]) / file_length, 4))
            bdf_data_1.append(1 - round(len(all_intern_columns_2[iDensityPoint - blank_space_counter][column_names[iDensityPoint - blank_space_counter]]) / file_length, 4))

    # Newton-type
    blank_space_counter = 0
    for iDensityPoint in range(int(len(correct_files_1)/2), len(correct_files_1) + 4):
        if iDensityPoint in [int(len(correct_files_1)/2) + 7, int(len(correct_files_1)/2) + 15, int(len(correct_files_1)/2) + 23, int(len(correct_files_1)/2) + 31]:
            adams_data_2.append(math.inf)
            bdf_data_2.append(math.inf)
            blank_space_counter += 1
        else:
            adams_data_2.append(1 - round(len(all_intern_columns_1[iDensityPoint - blank_space_counter][column_names[iDensityPoint - blank_space_counter]]) / file_length, 4))
            bdf_data_2.append(1 - round(len(all_intern_columns_2[iDensityPoint - blank_space_counter][column_names[iDensityPoint - blank_space_counter]]) / file_length, 4))

    return adams_data_1, bdf_data_1, adams_data_2, bdf_data_2

# call functions
am_fn, bdf_fn, am_nt, bdf_nt = load_data()


Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data 

In [26]:
am_fn = np.array(am_fn)
bdf_fn = np.array(bdf_fn)
am_nt = np.array(am_nt)
bdf_nt = np.array(bdf_nt)

# remove the infs
am_fn = am_fn[np.isfinite(am_fn)]
bdf_fn = bdf_fn[np.isfinite(bdf_fn)]
am_nt = am_nt[np.isfinite(am_nt)]
bdf_nt = bdf_nt[np.isfinite(bdf_nt)]

am = np.array([*am_fn, *am_nt])
bdf = np.array([*bdf_fn, *bdf_nt])

## Exact Fisher tests

In [34]:
def fisher_test(arr1, arr2):
    table = [[sum(arr1 < arr2), sum(arr1 >= arr2)],
             [sum(arr2 < arr1), sum(arr2 >= arr1)]]
    print(table)
    print(stats.fisher_exact(table))

fisher_test(am_fn, bdf_fn)
fisher_test(am_nt, bdf_nt)

fisher_test(am_fn, am_nt)
fisher_test(bdf_fn, bdf_nt)

fisher_test(am, bdf)

[[0, 35], [29, 6]]
(0.0, 8.015932228980983e-14)
[[1, 34], [28, 7]]
(0.007352941176470588, 1.1703261054312211e-11)
[[8, 27], [27, 8]]
(0.0877914951989026, 1.0730310749222526e-05)
[[7, 28], [28, 7]]
(0.0625, 8.550419756626659e-07)
[[1, 69], [57, 13]]
(0.0033053648614289346, 5.533319277324331e-25)


In [45]:
# 1 is functional, 2 is Newton

def load_data():        
    # check whether the folder 'Benchmarking_of_numerical_ODE_integration_methods/Data' exists
    if not os.path.exists('../../Benchmarking_of_numerical_ODE_integration_methods/Data/WholeStudy'):
        base_path = '../Data/WholeStudy'
    elif os.path.exists('../../Benchmarking_of_numerical_ODE_integration_methods/Data/WholeStudy'):
        base_path = '../../Benchmarking_of_numerical_ODE_integration_methods/Data/WholeStudy'

    # list of all data frames for nonLinSol == 1 for better indexing in the future
    all_intern_columns_1 = [pd.DataFrame(columns=[]) for _ in range(70)]

    # list of all data frames for nonLinSol == 2 for better indexing in the future
    all_intern_columns_2 = [pd.DataFrame(columns=[]) for _ in range(70)]

    all_intern_columns = [all_intern_columns_1, all_intern_columns_2]
    column_names = []

    # choose only the correct files
    all_files = sorted(os.listdir(base_path))
    correct_files_1 = []
    correct_files_2 = []
    for iFile in range(0, len(all_files)):
        if all_files[iFile].split('_')[0] == '1':
            correct_files_1.append(all_files[iFile])
        elif all_files[iFile].split('_')[0] == '2':
            correct_files_2.append(all_files[iFile])
    correct_files = [correct_files_1, correct_files_2]

    # open all .tsv linear solver files + save right column in data frame
    for iNonLinSol in range(0, len(correct_files)):
        for iCorrectFile in range(0, len(correct_files_1)):
            next_tsv = pd.read_csv(base_path + '/' + correct_files[iNonLinSol][iCorrectFile], sep='\t')

            # change .tsv-id form e.g. 1_06_10.tsv to 06_10
            new_name = correct_files[iNonLinSol][iCorrectFile].split('.')[0].split('_')[3] + '_' + \
                       correct_files[iNonLinSol][iCorrectFile].split('.')[0].split('_')[4]

            # reset after each iteration
            next_time_value = []
            num_x = []

            # open next file
            next_tsv = averaging(next_tsv)

            # get the correct values
            for iFile in range(0, len(next_tsv['id'])):  # each file
                if next_tsv['t_intern_ms'][iFile] != 0:
                    next_time_value.append(next_tsv['t_intern_ms'][iFile])
                    num_x.append(next_tsv['state_variables'][iFile])

            # append new column to existing data frame with correct values
            column_names.append(str(new_name))
            all_intern_columns[iNonLinSol][iCorrectFile]['state_variables'] = pd.Series(num_x)
            all_intern_columns[iNonLinSol][iCorrectFile][str(new_name)] = pd.Series(next_time_value)            

    # length of the last file
    total = len(next_tsv['id'])

    # initialize y-data
    am_fn_nr = []
    bdf_fn_nr = []
    am_nt_nr = []
    bdf_nt_nr = []

    # Functional
    for iDensityPoint in range(0, int(len(correct_files_1)/2)):
        am_fn_nr.append(len(all_intern_columns_1[iDensityPoint][column_names[iDensityPoint]]))
        bdf_fn_nr.append(len(all_intern_columns_2[iDensityPoint][column_names[iDensityPoint]]))

    # Newton-type
    for iDensityPoint in range(int(len(correct_files_1)/2), len(correct_files_1)):
        am_nt_nr.append(len(all_intern_columns_1[iDensityPoint][column_names[iDensityPoint]]))
        bdf_nt_nr.append(len(all_intern_columns_2[iDensityPoint][column_names[iDensityPoint]]))
        
    # to numpy
    am_fn_nr = np.array(am_fn_nr)
    bdf_fn_nr = np.array(bdf_fn_nr)
    am_nt_nr = np.array(am_nt_nr)
    bdf_nt_nr = np.array(bdf_nt_nr)
    
    #m_fn_fail_frac = (1 - am_fn_nr / total)
    #bdf_fn_fail_frac = (1 - bdf_fn_nr / total)
    #am_nt_fail_frac = (1 - am_nt_nr / total)
    #bdf_nt_fail_frac = (1 - bdf_nt_nr / total)

    return am_fn_nr, bdf_fn_nr, am_nt_nr, bdf_nt_nr, total

# call functions
am_fn_nr, bdf_fn_nr, am_nt_nr, bdf_nt_nr, total = load_data()

Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data frame!
Only 4 iterations needed for correct new data 

In [50]:
am_nr = np.array([*am_fn_nr, *am_nt_nr])
bdf_nr = np.array([*bdf_fn_nr, *bdf_nt_nr])

fn_nr = np.array([*am_fn_nr, *bdf_fn_nr])
nt_nr = np.array([*am_nt_nr, *bdf_nt_nr])

In [51]:
def fisher_test(arr1, arr2):
    n_success1 = sum(arr1)
    n_failure1 = len(arr1) * total - n_success1
    n_success2 = sum(arr2)
    n_failure2 = len(arr2) * total - n_success2
    table = [[n_success1, n_failure1],
             [n_success2, n_failure2]]
    print(table)
    print(stats.fisher_exact(table))

print("AM vs BDF, for Functional & Newton")
fisher_test(am_fn_nr, bdf_fn_nr)
fisher_test(am_nt_nr, bdf_nt_nr)

print("Functional vs Newton, for AM & BDF")
fisher_test(am_fn_nr, am_nt_nr)
fisher_test(bdf_fn_nr, bdf_nt_nr)

print("AM vs BDF")
fisher_test(am_nr, bdf_nr)

print("Functional vs Newton")
fisher_test(fn_nr, nt_nr)

AM vs BDF, for Functional & Newton
[[5161, 684], [5215, 630]]
(0.9115153655951961, 0.12065421897392267)
[[5293, 552], [5482, 363]]
(0.6349366702092223, 8.335102425427889e-11)
Functional vs Newton, for AM & BDF
[[5161, 684], [5293, 552]]
(0.7868916576345455, 8.033693572141634e-05)
[[5215, 630], [5482, 363]]
(0.548127204183388, 7.097524427121602e-19)
AM vs BDF
[[10454, 1236], [10697, 993]]
(0.7851475461317073, 6.899821276195114e-08)
Functional vs Newton
[[10376, 1314], [10775, 915]]
(0.6705611882740573, 6.427394861309469e-19)
